In [ ]:
"""

import os
import requests
import pandas as pd
from datetime import datetime
from dateutil.relativedelta import relativedelta
from tqdm import tqdm

# API setup
API = "https://planningdata.london.gov.uk/api-guest/applications/_search"
HDR = {"X-API-AllowRequest": "be2rmRnt&", "Content-Type": "application/json"}

# Boroughs list
BOROUGHS = [
    "City of London", "Barking and Dagenham", "Barnet", "Bexley", "Brent", "Bromley",
    "Camden", "Croydon", "Ealing", "Enfield", "Greenwich", "Hackney",
    "Hammersmith and Fulham", "Haringey", "Harrow", "Havering", "Hillingdon",
    "Hounslow", "Islington", "Kensington and Chelsea", "Kingston upon Thames",
    "Lambeth", "Lewisham", "Merton", "Newham", "Redbridge", "Richmond upon Thames",
    "Southwark", "Sutton", "Tower Hamlets", "Waltham Forest", "Wandsworth", "Westminster"
]

START_YEAR, END_YEAR = 2010, 2025

# Output path
OUT_PATH = r"c:/Users/Jose Miguel/OneDrive/Ambiente de Trabalho/NOVA/TESE/all_planning_data.parquet"

# Fields to pull
SOURCE_FIELDS = [
    "id", "lpa_app_no", "case_reference",
    "lpa_name", "ward_name", "postcode",
    "received_date", "valid_date", "decision_date", "appeal_date", "last_updated",
    "application_type", "development_type", "status", "case_officer",
    "decision", "decision_level", "planning_portal_reference",
    "residential_units.proposed", "residential_units.existing", "residential_units.net"
]

def year_iter(y0, y1):
    for y in range(y0, y1 + 1):
        gte = f"01/01/{y}"
        lt = f"01/01/{y+1}"
        yield gte, lt, y

def fetch_borough_year(borough: str, gte_date: str, lt_date: str, size=1000):
    all_rows = []
    page = 0
    while True:
        query = {
            "query": {
                "bool": {
                    "must": [
                        {"term": {"lpa_name.raw": borough}},
                        {"range": {"valid_date": {"gte": gte_date, "lt": lt_date}}}
                    ]
                }
            },
            "_source": True,
            "from": page * size,
            "size": size
        }
        resp = requests.post(API, headers=HDR, json=query, timeout=60)
        if resp.status_code >= 400:
            raise RuntimeError(f"HTTP {resp.status_code}: {resp.text[:300]}")
        payload = resp.json()
        hits = payload.get("hits", {}).get("hits", [])
        if not hits:
            break
        all_rows.extend([h.get("_source", {}) for h in hits])
        if len(hits) < size:
            break
        page += 1
    return all_rows

def main():
    all_data = []
    for b in BOROUGHS:
        print(f"\n== {b} ==")
        for gte, lt, year in tqdm(list(year_iter(START_YEAR, END_YEAR)), desc="Years"):
            try:
                rows = fetch_borough_year(b, gte, lt)
                all_data.extend(rows)
                print(f"Fetched {len(rows)} records for {b} {year}")
            except Exception as e:
                print(f"[ERR] {b} {year}: {e}")

    print("\nConverting to DataFrame...")
    df = pd.DataFrame(all_data)

    # Parse dates with UK style
    for col in ["received_date", "valid_date", "decision_date", "appeal_date", "last_updated"]:
        if col in df.columns:
            df[col] = pd.to_datetime(df[col], errors="coerce", dayfirst=True)

    print(f"Saving {len(df)} rows to {OUT_PATH}")
    df.to_parquet(OUT_PATH, index=False, engine="pyarrow")

if __name__ == "__main__":
    main()
"""


== City of London ==


Years:  12%|█▎        | 2/16 [00:00<00:03,  3.89it/s]

Fetched 39 records for City of London 2010
Fetched 34 records for City of London 2011


Years:  25%|██▌       | 4/16 [00:00<00:02,  5.37it/s]

Fetched 72 records for City of London 2012
Fetched 63 records for City of London 2013


...

Fetched 8904 records for Westminster 2024


Years: 100%|██████████| 16/16 [00:15<00:00,  1.05it/s]

Fetched 8489 records for Westminster 2025

Converting to DataFrame...



C:\Users\Jose Miguel\AppData\Local\Temp\ipykernel_41512\3135013865.py:91: UserWarning: Parsing dates in %Y-%m-%dT%H:%M:%S.%f%z format when dayfirst=True was specified. Pass `dayfirst=False` or specify a format to silence this warning.
  df[col] = pd.to_datetime(df[col], errors="coerce", dayfirst=True)
C:\Users\Jose Miguel\AppData\Local\Temp\ipykernel_41512\3135013865.py:91: FutureWarning: In a future version of pandas, parsing datetimes with mixed time zones will raise an error unless `utc=True`. Please specify `utc=True` to opt in to the new behaviour and silence this warning. To create a `Series` with mixed offsets and `object` dtype, please use `apply` and `datetime.datetime.strptime`
  df[col] = pd.to_datetime(df[col], errors="coerce", dayfirst=True)


Saving 803454 rows to c:/Users/Jose Miguel/OneDrive/Ambiente de Trabalho/NOVA/TESE/all_planning_data.parquet


In [6]:
import requests
import pandas as pd
from tqdm import tqdm
from datetime import datetime

# ==============================
# API CONFIG
# ==============================
API_URL = "https://planningdata.london.gov.uk/api-guest/applications/_search"
HEADERS = {
    "X-API-AllowRequest": "be2rmRnt&",
    "Content-Type": "application/json"
}

BOROUGHS = [
    "City of London", "Barking and Dagenham", "Barnet", "Bexley", "Brent", "Bromley",
    "Camden", "Croydon", "Ealing", "Enfield", "Greenwich", "Hackney",
    "Hammersmith and Fulham", "Haringey", "Harrow", "Havering", "Hillingdon",
    "Hounslow", "Islington", "Kensington and Chelsea", "Kingston upon Thames",
    "Lambeth", "Lewisham", "Merton", "Newham", "Redbridge", "Richmond upon Thames",
    "Southwark", "Sutton", "Tower Hamlets", "Waltham Forest", "Wandsworth", "Westminster"
]

START_YEAR = 2010
END_YEAR = 2025

OUT_PATH = r"c:/Users/Jose Miguel/OneDrive/Ambiente de Trabalho/NOVA/TESE/all_planning_data.parquet"


# ==============================
# HELPERS
# ==============================
def year_ranges(start, end):
    for y in range(start, end + 1):
        yield f"01/01/{y}", f"01/01/{y+1}", y


def fetch_borough_year(borough, gte, lt, size=1000):
    rows = []
    page = 0

    while True:
        query = {
            "query": {
                "bool": {
                    "must": [
                        {"term": {"lpa_name.raw": borough}},
                        {"range": {"valid_date": {"gte": gte, "lt": lt}}}
                    ]
                }
            },
            "_source": True,   # IMPORTANT: PULL EVERYTHING
            "from": page * size,
            "size": size
        }

        r = requests.post(API_URL, headers=HEADERS, json=query, timeout=60)
        r.raise_for_status()

        hits = r.json().get("hits", {}).get("hits", [])
        if not hits:
            break

        rows.extend([h["_source"] for h in hits])
        if len(hits) < size:
            break

        page += 1

    return rows


# ==============================
# MAIN
# ==============================
def main():
    all_rows = []

    for borough in BOROUGHS:
        print(f"\n=== {borough} ===")
        for gte, lt, year in tqdm(list(year_ranges(START_YEAR, END_YEAR)), desc="Years"):
            try:
                data = fetch_borough_year(borough, gte, lt)
                all_rows.extend(data)
                print(f"{year}: {len(data)} records")
            except Exception as e:
                print(f"[ERROR] {borough} {year}: {e}")

    print("\nCreating DataFrame...")
    df = pd.DataFrame(all_rows)

    # ==============================
    # DATE PARSING
    # ==============================
    date_cols = [
        "received_date", "valid_date", "decision_date",
        "appeal_date", "last_updated", "lapsed_date",
        "commencement_date"
    ]

    for col in date_cols:
        if col in df.columns:
            df[col] = pd.to_datetime(df[col], errors="coerce", dayfirst=True)

    # ==============================
    # NORMALISE RESIDENTIAL UNITS
    # ==============================
    if "residential_units" in df.columns:
        print("Normalising residential_units...")
        resi = pd.json_normalize(df["residential_units"])
        resi.columns = [f"residential_units_{c}" for c in resi.columns]
        df = pd.concat([df.drop(columns=["residential_units"]), resi], axis=1)

    # ==============================
    # KEEP ONLY ANALYTICAL COLUMNS
    # ==============================
    KEEP_COLS = [
        "id", "lpa_name", "ward_name", "postcode",
        "received_date", "valid_date", "decision_date",
        "lapsed_date", "commencement_date",
        "application_type", "development_type",
        "decision", "status", "decision_level",
        "case_officer", "planning_portal_reference",
        "residential_units_existing",
        "residential_units_proposed",
        "residential_units_net"
    ]

    KEEP_COLS = [c for c in KEEP_COLS if c in df.columns]
    df = df[KEEP_COLS]

    print(f"Final dataset shape: {df.shape}")
    print("Saving to parquet...")
    df.to_parquet(OUT_PATH, engine="pyarrow", index=False)

    print("DONE")


if __name__ == "__main__":
    main()



=== City of London ===


Years:  12%|█▎        | 2/16 [00:00<00:02,  5.74it/s]

2010: 39 records
2011: 34 records


Years:  25%|██▌       | 4/16 [00:00<00:02,  5.61it/s]

2012: 72 records
2013: 63 records


...

2024: 8904 records


Years: 100%|██████████| 16/16 [00:25<00:00,  1.58s/it]

2025: 8489 records

Creating DataFrame...



C:\Users\Jose Miguel\AppData\Local\Temp\ipykernel_41512\4188901887.py:103: UserWarning: Parsing dates in %Y-%m-%dT%H:%M:%S.%f%z format when dayfirst=True was specified. Pass `dayfirst=False` or specify a format to silence this warning.
  df[col] = pd.to_datetime(df[col], errors="coerce", dayfirst=True)
C:\Users\Jose Miguel\AppData\Local\Temp\ipykernel_41512\4188901887.py:103: FutureWarning: In a future version of pandas, parsing datetimes with mixed time zones will raise an error unless `utc=True`. Please specify `utc=True` to opt in to the new behaviour and silence this warning. To create a `Series` with mixed offsets and `object` dtype, please use `apply` and `datetime.datetime.strptime`
  df[col] = pd.to_datetime(df[col], errors="coerce", dayfirst=True)


Final dataset shape: (803454, 10)
Saving to parquet...
DONE


In [7]:
import pandas as pd

pd.set_option("display.max_columns", None)
pd.set_option("display.width", 2000)

df = pd.read_parquet(r"c:/Users/Jose Miguel/OneDrive/Ambiente de Trabalho/NOVA/TESE/all_planning_data.parquet")
print(df.head())
print("N colunas:", df.shape[1])
print(df.columns.tolist())


                             id        lpa_name  postcode valid_date decision_date lapsed_date application_type development_type  decision    status
0  City_of_London-10_00541_FULL  City of London  EC2M 3TQ 2010-07-27    2010-08-26  2013-08-26        All Other             None  Approved  Approved
1   City_of_London-10_00063_LBC  City of London      None 2010-02-19    2010-11-11  2013-11-11        All Other             None  Approved  Approved
2  City_of_London-10_00071_FULL  City of London  EC2N 2HA 2010-03-04    2010-04-29  2013-04-29        All Other             None  Approved  Approved
3  City_of_London-10_00846_FULL  City of London  EC4N 8BN 2010-12-01    2011-01-27  2014-01-27        All Other             None  Approved  Approved
4   City_of_London-10_00078_MDC  City of London      None 2010-02-22    2011-05-19  2016-05-19        All Other             None  Approved  Approved
N colunas: 10
['id', 'lpa_name', 'postcode', 'valid_date', 'decision_date', 'lapsed_date', 'application_ty

In [5]:
import requests
import pandas as pd

API = "https://planningdata.london.gov.uk/api-guest/applications/_search"
HDR = {"X-API-AllowRequest": "be2rmRnt&", "Content-Type": "application/json"}

q = {
  "query": {
    "bool": {
      "must": [
        {"term": {"lpa_name.raw": "City of London"}},
        {"range": {"valid_date": {"gte": "01/01/2010", "lt": "01/01/2011"}}}
      ]
    }
  },
  "_source": True,   # <-- traz tudo
  "size": 1
}

r = requests.post(API, headers=HDR, json=q, timeout=60)
r.raise_for_status()
hit = r.json()["hits"]["hits"][0]["_source"]

print("Nº chaves no _source:", len(hit.keys()))
print(sorted(hit.keys()))
print("\nExemplo completo (primeiras chaves):")
print(dict(list(hit.items())[:30]))


Nº chaves no _source: 52
['actual_commencement_date', 'actual_completion_date', 'appeal_decision', 'appeal_decision_date', 'appeal_start_date', 'appeal_status', 'application_details', 'application_type', 'application_type_full', 'bo_system', 'borough', 'centroid', 'centroid_easting', 'centroid_northing', 'cil_liability', 'date_building_work_completed_under_previous_permission', 'date_building_work_started_under_previous_permission', 'decision', 'decision_agency', 'decision_conditions', 'decision_date', 'decision_process', 'decision_target_date', 'description', 'development_type', 'epc_number', 'id', 'lapsed_date', 'last_date_consultation_comments', 'last_synced', 'last_updated', 'last_updated_by', 'locality', 'lpa_app_no', 'lpa_name', 'parking_details', 'polygon', 'postcode', 'pp_id', 'reference_no_of_permission_being_relied_on', 'secondary_street_name', 'site_name', 'site_number', 'status', 'street_name', 'subdivision_of_building', 'title_number', 'uprn', 'url_planning_app', 'valid_da

In [2]:
from ydata_profiling import ProfileReport
profile_students = ProfileReport(df, title='API Data', explorative=True)
profile_students.to_notebook_iframe()

Summarize dataset:   0%|          | 0/5 [00:00<?, ?it/s]

Generate report structure:   0%|          | 0/1 [00:00<?, ?it/s]

Render HTML:   0%|          | 0/1 [00:00<?, ?it/s]

In [4]:
df = pd.read_parquet(r"c:/Users/Jose Miguel/OneDrive/Ambiente de Trabalho/NOVA/TESE/all_planning_data.parquet")

# Take a random sample of 5,000 rows (you can change this number)
sample = df.sample(5000, random_state=42)

# Save to CSV
sample.to_csv(r"c:/Users/Jose Miguel/OneDrive/Ambiente de Trabalho/NOVA/TESE/sample_planning_data.csv", index=False, encoding="utf-8")

print("Sample saved!")

Sample saved!
